In [91]:
import os
import pandas as pd

import netpixi
import graph_tool_extras as gte

from graph_tool import draw

path = "../__shared__/premier-league.csv"
data = pd.read_csv(path)
# data.head(10)


In [92]:
data_filtered = data.query('year >= 2020')
data_filtered = data_filtered.sort_values(by='fee_cleaned', ascending=False)
data_filtered = data_filtered.query('transfer_movement != "out"')
# data_filtered.head()

In [93]:
filt = data_filtered.iloc[:,[0,4]] #club name é quem recebeu, club involved name é quem enviou
total = filt.values.tolist()
# total

In [94]:
lista_times = []

for trans in total:
    if trans[0] not in lista_times:
        lista_times.append(trans[0])
    if trans[1] not in lista_times:
        lista_times.append(trans[1])

print(len(lista_times))
print(trans)

330
['Wolverhampton Wanderers', 'Grasshoppers']


In [95]:
import graph_tool_extras as gte

In [107]:
g = gte.Graph(directed=True) # DIFERENTE DE gte.Graph(), pois directed é falso por padrão

In [108]:
g.add_vp('transacoes')
g.add_vp('envios')
g.add_vp('recebimentos')
# g.add_ep('total')

In [109]:
def get_or_add_vertex(g, id):
    u = g.vertex_by_id(id)
    if u is None:
        u = g.add_vertex_by_id(id)
        u['transacoes'] = 0
        u['envios'] = 0
        u['recebimentos'] = 0
    return u

In [110]:
def get_or_add_edge(g, t1, t0):
    e = g.edge_by_ids(t1, t0)
    if e is None:
        e = g.add_edge_by_ids(t1, t0)
        # e['total'] = 0
        u1 = g.vertex_by_id(t1)
        u2 = g.vertex_by_id(t0)
        u1['transacoes'] += 1
        u2['transacoes'] += 1
        u1['envios'] += 1
        u2['recebimentos'] += 1
        
    return e

In [111]:
for team in lista_times:
    get_or_add_vertex(g,team)
    # g.add_vertex_by_id(team)

In [112]:
for trans in total:
    get_or_add_edge(g,trans[1],trans[0])
    # g.add_edge_by_ids(trans[1], trans[0])

In [113]:
# gte.save(g, 'teste1.net.gz')

In [114]:
# netpixi.render('../__shared__/teste1.net.gz');

In [115]:
layout = draw.sfdp_layout(g)

In [116]:
gte.move(g, layout)

In [117]:
gte.save(g, 'entrega1.net.gz')

In [118]:
r = netpixi.render('entrega1.net.gz', infinite=True)

In [119]:
r.vertex_set_key('transacoes')

In [120]:
r.vertex_set_key('envios')

In [121]:
r.vertex_set_key('recebimentos')

In [122]:
r.vertex_set_key('id')

In [123]:
r.vertex_unset_key()

In [124]:
r.vertex_default(size=8, bwidth=1)

In [125]:
r.edge_default(width=0.5)

In [126]:
print(f"Número de vértices: {g.num_vertices()}")
print(f"Número de arestas: {g.num_edges()}")

Número de vértices: 330
Número de arestas: 751
